In [195]:
import numpy as np 
import pandas as pd 

from matminer.featurizers.site import LocalPropertyDifference
from matminer.featurizers.structure import SiteStatsFingerprint
from matminer.datasets.dataframe_loader import load_elastic_tensor
from matminer.featurizers.base import MultipleFeaturizer, BaseFeaturizer
from matminer.featurizers.stats import PropertyStats
from matminer.utils.data import MagpieData

class VBS_mean(BaseFeaturizer):
    def __init__(self, properties, stats=('mean', 'std_dev'), deviation='AD'):
        self.properties = properties
        self.stats = tuple([stats]) if type(stats) == str else stats
        self.deviation = tuple([deviation]) if type(deviation) == str else deviation
        if self.stats and '_mode' in ''.join(self.stats):
            nmodes = 0
            for stat in self.stats:
                if '_mode' in stat and int(stat[0]) > nmodes:
                    nmodes = int(stat[0])
            self.nmodes = nmodes
            
    @staticmethod
    def from_preset(preset):
        """
        Create a new LocalPropertyDifference class according to a preset

        Args:
            preset (str) - Name of preset
        """
        if preset == "dejong2016":
            return (VBS_mean(
                properties=["Number", "AtomicWeight",
                            "Column", "Row", "CovalentRadius",
                            "Electronegativity"], 
                stats=['holder_mean::%d'%d for d in range(0, 4+1)] + ['geom_std_dev'],
                deviation='AD'
                ), 
            VBS_mean(
                properties=["Number", "AtomicWeight",
                            "Column", "Row", "CovalentRadius",
                            "Electronegativity"], 
                stats=['holder_mean::%d'%d for d in [1, 2, 4]] + ['geom_std_dev'],
                deviation='SD'
                ))    
        else:
            raise ValueError('Unrecognized preset: ' + preset)
    
    def featurize(self, strc):
        ft = LocalPropertyDifference(properties=self.properties)
        _site_labels = ft.feature_labels()
        print ("_site_labels: ", _site_labels)
        vals = []
        for dev in self.deviation:
            if dev == "AD":
                vals_AD = [[] for t in _site_labels]
                for idx, site in enumerate(strc.sites):
                    opvalstmp = ft.featurize(strc, idx)
                    for j, opval in enumerate(opvalstmp):
                        if opval is None:
                            vals_AD[j].append(0.0)
                        else:
                            vals_AD[j].append(opval)
                vals += vals_AD

            elif dev == "SD":
                vals_SD = [[] for t in _site_labels]
                for idx, site in enumerate(strc.sites):
                    # Get the targeted site
                    my_site = strc[idx]

                    # Get the tessellation of a site
                    nn = get_nearest_neighbors(VoronoiNN(weight='area'), strc, idx)

                    # Get the element and weight of each site
                    elems = [n['site'].specie for n in nn]
                    weights = [n['weight'] for n in nn]

                    # Compute the difference for each property
                    opvalstmp = np.zeros((len(self.properties),))
                    total_weight = np.sum(weights)
                    for i,p in enumerate(self.properties):
                        my_prop = MagpieData().get_elemental_property(my_site.specie, p)
                        n_props = MagpieData().get_elemental_properties(elems, p)
                        opvalstmp[i] = np.dot(weights, np.subtract(n_props, my_prop)) / total_weight
                    for j, opval in enumerate(opvalstmp):
                        if opval is None:
                            vals_SD[j].append(0.0)
                        else:
                            vals_SD[j].append(opval)
                vals += vals_SD
            else:
                raise ValueError('Unrecognized mean difference type ' + dev)
        if self.stats:
            stats = []
            for op in vals:
                for stat in self.stats:
                    stats.append(PropertyStats().calc_stat(op, stat))

            return stats
        else:
            return vals
    
    def feature_labels(self):
        if self.stats:
            labels = []
            for dev in self.deviation:
                for prop in self.properties:
                    for stat in self.stats:
                        labels.append("%s %s %s" %(dev, stat, prop))
            return labels
        else:
            return LocalPropertyDifference(properties=self.properties).feature_labels()

In [200]:
ft = VBS_mean.from_preset("dejong2016")
d = ft.featurize(c)
print (ft.feature_labels())
d

AttributeError: 'tuple' object has no attribute 'featurize'